## 1. Overview of GCG

[GCG (Gradient-based Candidate Generation)](https://arxiv.org/abs/2307.15043) is a sophisticated jailbreaking attack method based on optimization algorithms. It aims to exploit the vulnerabilities of language models by intelligently manipulating the tokens in the `jailbreak_prompt`. By calculating gradients and iteratively replacing tokens, GCG minimizes the loss in the `reference_response` to craft prompts that can induce unexpected model behaviors.

## 2. Detailed Explanation of the GCG Method

The GCG method operates by focusing on the `jailbreak_prompt` tokens. It involves the following steps:

- **Gradient Calculation:** Computes the gradient of the one-hot vectors of tokens in the `jailbreak_prompt`.
- **Token Replacement:** Randomly selects a position in `jailbreak_prompt` and replaces the token at that position with one from the `top_k` tokens with the smallest gradient values.
- **Sample Generation and Selection:** Generates `num_turb_sample` new samples and selects the one with the lowest loss in the `reference_response`.
- **Iteration and Convergence:** Repeats the process for up to `max_num_iter` iterations or until the `target_response` prefix matches the `reference_response` exactly.

## 3. How to Use GCG?
The following steps guide you through setting up your environment, preparing the dataset, initiating the GCG attack, and saving the results. Each step is associated with corresponding code snippets, which you should run in your Jupyter Notebook cells.

### 3.1 Setting up the Environment

Before starting, ensure that you have installed the necessary packages, set up logging, and seeded the environment for reproducibility.

The logging level is set up to be `INFO`. It can be changed to `DEBUG` for more details of the inner progress.

In [ ]:
import easyjailbreak
import torch
import logging
from transformers import set_seed

# Set up logging format and seed
logging.basicConfig(level=logging.INFO, format='[%(asctime)s] {%(filename)s:%(lineno)d} %(levelname)s - %(message)s')
set_seed(42)

### 3.2 Preparing the Dataset and Model
Load model, tokenizer, and dataset. The dataset is sliced to the first 10 entries for demonstration purposes. Adjust the slice as needed for your use case.

The `easyjailbreak.models.from_pretrained` method provides a convenient, one-click feature to load Hugging Face models. It returns an instance of the `easyjailbreak.models.HuggingfaceModel` class. This class packages the model and tokenizer together, ensuring unified, convenient management and use.

The parameter `'vicuna_v1.1'` might catch your attention. This is used to indicate the specific version or type of the model. Based on this parameter, the method employs the `fastchat` library to obtain specific conversational formats tailored for the model.

For more information about the possible model options and templates, and to understand how you can leverage them for your specific use-case, refer to the `FastChat` library's `conversation.py`. This resource is a treasure trove of information on different conversational formats and how they can be applied: [FastChat conversation.py](https://github.com/lm-sys/FastChat/blob/main/fastchat/conversation.py)

In [ ]:
# Load model and tokenizer
model = easyjailbreak.models.from_pretrained(
    'lmsys/vicuna-7b-v1.5', 
    'vicuna_v1.1', 
    dtype=torch.bfloat16, 
    max_new_tokens=40)

# Load dataset
dataset_name = 'AdvBench'
dataset = easyjailbreak.datasets.JailbreakDataset(dataset_name)[:10]

### 3.3 Initiating and launching the Attack
Initialize the API class with the required settings and start the attack process. Adjust the `batchsize` if you encounter Out of Memory (OOM) issues.

In [ ]:
# API class initialization and settings
attacker = easyjailbreak.attacker.GCG(
    attack_model=model,
    target_model=model,
    jailbreak_datasets=dataset,
    jailbreak_prompt_length=20,
    num_turb_sample=512,
    batchsize=256,      # decrease if OOM
    top_k=256,
    max_num_iter=500
)

# Launch the attack
attacker.attack()

### 3.4 Saving the Results
After the attack, save the results for analysis and review.

In [ ]:
output_file = 'gcg_result.jsonl'
attacker.jailbreak_datasets.save_to_jsonl(output_file)

### 4. Additional Resources
The provided example/run_gcg.py script demonstrates a complete workflow, from setting up the environment to saving the attack results. It showcases how to use the GCG method effectively with the easyjailbreak package.